In [ ]:
from github import Github
import pandas as pd 
import itertools
import json
import requests

#### Insert personal access token.

In [ ]:
%store -r access_token
g = Github(access_token)

##### These variables are global variables 

In [ ]:
repo_var = None
page_num=None
repo_list=list()
list_without_lesson=list()
list_with_lesson=list()
final_repo_list=list()

#### Declaring a function

* To get the total repositories count
* Creating a page_num variable to hold the valid page numbers to iterate the repo list.
* Final repo list created from repo list
* List with lesson created from final repo list
* List without lesson created from final repo list

In [ ]:
# Declaring the function.

def repo_initialize():
    
    '''
    Function takes no paramter.
    Returning total repositories count, final repository list, list_with_lesson & list_without_lesson.
    
    '''
    
    global repo_var
    global page_num
    global repo_list
    global list_without_lesson
    global list_with_lesson
    global final_repo_list
    
    repo_var = g.get_organization("carpentries-incubator").get_repos()

    page_num = 0
    for i in range(0,10):
        if len(repo_var.get_page(i))==0:
            page_num = i
            break
        else:
            pass
    repo_list = list()
    for i in range(0,page_num):
        repo_list.append(repo_var.get_page(i))
    final_repo_list = list((itertools.chain.from_iterable(repo_list)))

# Repository lists. 1)list_with_lesson & list_without_lesson.

    for repo in final_repo_list:
        tags=repo.get_topics()
        if "lesson" in tags:
            list_with_lesson.append(repo)
        else:
            list_without_lesson.append(repo)

#Calling the function

repo_initialize()


#### Declaring functions to get  following information of all the repsoitories stored in list_with_lesson.

* issue Count
* Pull Request Count
* Stargazers Count

In [ ]:
#### Dataframe column heading

cols=["repo","open_PR", "closed_PR","closed_issues","open_issues","stargazers_count"]
df=pd.DataFrame(columns=cols)
df['repo']=list_with_lesson

#### Declaring a function to obtain issues_count.

def issues_count(list_with_lesson):
    
    '''
    Functions takes List_with_lesson as a parameter
    Return a list of open_issues_count, closed_issues_count.
    '''
    open_issues_count=[]
    closed_issues_count=[]
    
    for repo in list_with_lesson:
        open_issues=repo.get_issues(state='open').totalCount
        open_issues_count.append(open_issues)
        closed_issues=repo.get_issues(state='closed').totalCount
        closed_issues_count.append(closed_issues)
    return open_issues_count,closed_issues_count

#### Calling the function

open_issues_count,closed_issues_count=issues_count(list_with_lesson)

#### Declairing a function to get open & closed Pulls count

def pulls_count(list_with_lesson):
    
    '''
    Function takes list_with_lesson as a parameter
    Returning a list of open_pr_count & closed_pr_count.
    '''
    open_pr_count=[]
    closed_pr_count=[]
    
    for repo in list_with_lesson:
        open_pr=repo.get_pulls(state='open').totalCount
        open_pr_count.append(open_pr)
        closed_pr=repo.get_pulls(state='closed').totalCount
        closed_pr_count.append(closed_pr)
    return open_pr_count,closed_pr_count

#### Calling the function

open_pr_count,closed_pr_count=pulls_count(list_with_lesson)

#### Declairing a function to get stargazers count

def stars_count(list_with_lesson):
    
    '''
    Functions takes list_with_lesson as a parameter
    Returning a list of Stargazers for each repsoitory.
    '''
    stargazers=[]
    for repo in list_with_lesson:
        stars=repo.stargazers_count
        stargazers.append(stars)
    return stargazers

#### Calling the function

stargazers=stars_count(list_with_lesson)

#### Putting the information into dataframe.

df['stargazers_count']=stargazers
df['open_PR']=open_pr_count
df['closed_PR']=closed_pr_count
df['open_issues']=open_issues_count
df['closed_issues']=closed_issues_count

In [ ]:
df.style.set_properties(subset=['repo'], **{'width-min': '300px'})